In [ ]:
from typing import Any, Literal
import json, ast
from pprint import pprint


class Node:
    def __init__(self, node_id: int, action_id: int, name):
        self.id = node_id
        self.name = name
        self.aid = action_id

    @staticmethod
    def from_dict(node_id: int, dict_data: dict[str, Any]):
        return Node(node_id, dict_data["aid"], dict_data["name"])

    def to_dict(self):
        # id is get directly from the object, no need to put it in the dictionary
        return {"name": self.name, "aid": self.aid}


class Action:
    def __init__(self, action_id: int, name, action_type, data):
        self.id = action_id
        self.name = name
        self.type = action_type
        self.data = data

    @staticmethod
    def from_dict(action_id: int, data: dict[str, Any]):
        return Action(action_id, data["name"], data["type"], data["data"])

    def to_dict(self):
        # id is get directly from the object, no need to put it in the dictionary
        return {"name": self.name, "type": self.type, "data": self.data}


class Route:
    def __init__(self, node_id_start: int, node_id_end: int, rule):
        self.nid_start = node_id_start
        self.nid_end = node_id_end
        self.rule = rule

    def to_dict(self):
        return {"nid_start": self.nid_start, "nid_end": self.nid_end, "rule": self.rule}

    @staticmethod
    def from_dict(node_id_start: int, node_id_end: int, data: dict[str, Any]):
        return Route(node_id_start, node_id_end, data["rule"])


In [ ]:
class Flow:
    def __init__(self):
        self.nodes: dict[int, Node] = {}  # Dictionary of node IDs to Node objects
        self.actions: dict[int, Action] = {}  # Dictionary of action IDs to Action objects
        self.routes: dict[int, dict[int, Route]] = {}  # Dictionary of route IDs to Route objects
        self.next_node_id = 1
        self.next_action_id = 1

    def add_action(self, action_name, action_type, action_data):
        action = Action(self.next_action_id, action_name, action_type, action_data)
        self.actions[action.id] = action
        self.next_action_id += 1
        return action

    def edit_action(self, action_id: int, new_name, new_type, new_data):
        action = self.actions.get(action_id)
        assert action is not None, "Action not found"
        action.name = new_name
        action.type = new_type
        action.data = new_data

    def delete_action(self, action_id: int):
        # check if the action is used in any node
        for node in self.nodes.values():
            if node.aid == action_id:
                raise Exception("Action is used in a node")
        self.actions.pop(action_id, None)

    def add_node(self, node_name, action_id: int):
        node = Node(self.next_node_id, action_id, node_name)
        self.nodes[node.id] = node
        self.next_node_id += 1
        return node

    def edit_node(self, node_id: int, new_name, new_action_id: int):
        node = self.nodes.get(node_id)
        assert node is not None, "Node not found"
        assert new_action_id in self.actions, "Invalid action ID"
        node.name = new_name
        node.aid = new_action_id

    def add_route(self, node_id_start: int, node_id_end: int, rule):
        assert node_id_start in self.nodes and node_id_end in self.nodes, "Invalid node ID"
        route = Route(node_id_start, node_id_end, rule)
        if node_id_start not in self.routes:
            self.routes[node_id_start] = {}
        self.routes[node_id_start][node_id_end] = route
        return route

    def edit_route(self, node_id_start: int, node_id_end: int, new_rule):
        assert node_id_start in self.nodes and node_id_end in self.nodes, "Invalid node ID"
        route = self.routes[node_id_start].get(node_id_end)
        assert route is not None, "Route not found"
        route.rule = new_rule

    def delete_route(self, node_id_start: int, node_id_end: int):
        self.routes[node_id_start].pop(node_id_end, None)
        if len(self.routes[node_id_start]) == 0:
            self.routes.pop(node_id_start, None)

    def delete_node(self, node_id: int):
        node = self.nodes.get(node_id)
        assert node is not None, "Node not found"
        self.nodes.pop(node_id, None)
        self.routes.pop(node_id, None)
        for node_id_end_routes in self.routes.values():
            node_id_end_routes.pop(node_id, None)  # all routes that goes to the deleted node

    def serialize(self, output_type: Literal["json", "python"] = "json"):
        data = {
            "nn": self.next_node_id,
            "na": self.next_action_id,
            "nodes": {node.id: node.to_dict() for node in self.nodes.values()},
            "actions": {action.id: action.to_dict() for action in self.actions.values()},
            "routes": {node_id_start: {node_id_end: route.to_dict()
                                       for node_id_end, route in node_id_end_routes.items()}
                       for node_id_start, node_id_end_routes in self.routes.items()}
        }
        # pprint(data, sort_dicts=False, indent=4)
        return json.dumps(data) if output_type == "json" else str(data)

    @staticmethod
    def deserialize(serialized_data: str, input_type: Literal["json", "python"] = "json"):
        data = json.loads(serialized_data) if input_type == "json" else ast.literal_eval(serialized_data)
        deserialized_flow = Flow()
        deserialized_flow.next_node_id = data["nn"]
        deserialized_flow.next_action_id = data["na"]

        for node_id_key, node_data in data["nodes"].items():
            node_id = int(node_id_key) if input_type == "json" else node_id_key
            deserialized_flow.nodes[node_id] = Node.from_dict(node_id, node_data)

        for action_id_key, action_data in data["actions"].items():
            action_id = int(action_id_key) if input_type == "json" else action_id_key
            deserialized_flow.actions[action_id] = Action.from_dict(action_id, action_data)

        for node_id_start, node_id_end_routes in data["routes"].items():
            node_id_start = int(node_id_start) if input_type == "json" else node_id_start
            deserialized_flow.routes[node_id_start] = {}
            for node_id_end, route_data in node_id_end_routes.items():
                node_id_end = int(node_id_end) if input_type == "json" else node_id_end
                deserialized_flow.routes[node_id_start][node_id_end] = Route.from_dict(node_id_start, node_id_end, route_data)

        return deserialized_flow

    def print_readable(self):
        print("Nodes:")
        for node in self.nodes.values():
            print(f"\tN{node.id}: {node.name} (Action ID: A{node.aid})")
        print("Actions:")
        for action in self.actions.values():
            print(f"\tA{action.id}: \n\t\tType: {action.type}\n\t\tData: {action.data}")
        print("Routes:")
        for node_id_end_routes in self.routes.values():
            for route in node_id_end_routes.values():
                print(f"\tFrom node {route.nid_start} to node {route.nid_end} (Rule: {route.rule})")


In [ ]:
flow = Flow()

action1 = flow.add_action("Action 1", "run-command", "test data")
action2 = flow.add_action("Action 2", "print-script", "test data")
action3 = flow.add_action("Action 3", "chat-with-bot", "test data")
action4 = flow.add_action("Action 4", "chat-with-human", "test data")
action5 = flow.add_action("Action 5", "search-items", "test data")

node1 = flow.add_node("node1", action1.id)
node2 = flow.add_node("node2", action2.id)
node3 = flow.add_node("node3", action3.id)
node4 = flow.add_node("node4", action5.id)

flow.add_route(node1.id, node2.id, "rule1")
flow.add_route(node1.id, node4.id, "rule5")
flow.add_route(node2.id, node1.id, "rule2")
flow.add_route(node2.id, node3.id, "rule3")
flow.add_route(node3.id, node1.id, "rule4")

serialized = flow.serialize()
print(serialized)
flow2 = Flow.deserialize(serialized)
flow2.print_readable()


In [ ]:
flow2.edit_action(1, "Action 1 edited", "run-command", "test data")
assert flow2.actions[1].name == "Action 1 edited"
assert flow2.actions[1].type == "run-command"
assert flow2.actions[1].data == "test data"

flow2.edit_node(1, "node1 edited", 2)
assert flow2.nodes[1].name == "node1 edited"
assert flow2.nodes[1].aid == 2

flow2.edit_route(1, 2, "rule1 edited")
assert flow2.routes[1][2].rule == "rule1 edited"

In [ ]:
flow2.print_readable()

In [ ]:
flow2.delete_action(1)
assert 1 not in flow2.actions

flow2.delete_route(3, 1)
assert 1 not in flow2.routes[3]

flow2.delete_node(1)
assert 1 not in flow2.nodes
assert 1 not in flow2.routes
for routes in flow2.routes.values():
    assert 1 not in routes

In [ ]:
flow2.print_readable()
serialized_flow = flow2.serialize()
serialized_flow

In [ ]:
flow3 = Flow.deserialize(serialized_flow)
print(flow3.serialize())